# New 02 - Task 01 - VetCyto - CutSplitStep.

# Training Step - Cutting Code.

In [ ]:
import json
from PIL import Image
from tqdm import tqdm
import os
import shutil

def crop_objects_from_image(image_path, json_path, output_dir):
    # Null address check
    if not (os.path.exists(image_path)):
        print("The jpeg path is empty! Check for mistakes! \ninput arguments: \n", "image_path =", image_path, "\n", "json_path =", json_path, "\n", "output_dir =", output_dir, "\n")
        return
    if not (os.path.exists(json_path)):
        print("The json path is empty! Check for mistakes! \ninput arguments: \n", "image_path =", image_path, "\n", "json_path =", json_path, "\n", "output_dir =", output_dir, "\n")
        return

    # Check if output directory exists and handle accordingly
    if not os.path.exists(output_dir):
        print(f"Creating output directory: '{output_dir}'")
        os.makedirs(output_dir, exist_ok=True)

    # Load the JSON file containing object annotations
    with open(json_path, 'r') as f:
        data = json.load(f)

    # Increase the maximum allowed image size by modifying the MAX_IMAGE_PIXELS constant in PIL to reduce zip bomb warnings
    Image.MAX_IMAGE_PIXELS = 250000000 

    # Open the image file
    image = Image.open(image_path)

    # Get the base name of the original image file (without extension)
    original_base_name = os.path.splitext(os.path.basename(image_path))[0]

    # Iterate through each shape in the JSON data, with a progress bar
    for shape in tqdm(data['shapes'], desc="Cropping objects"):
        points = shape['points']
        
        if shape['shape_type'] == 'rectangle':
            # Extract rectangle coordinates
            x1, y1 = points[0]
            x2, y2 = points[1]

            # Calculate the bounding box coordinates
            left = min(x1, x2)
            upper = min(y1, y2)
            right = max(x1, x2)
            lower = max(y1, y2)

        elif shape['shape_type'] == 'polygon':
            # Extract polygon coordinates
            x_coords = [point[0] for point in points]
            y_coords = [point[1] for point in points]

            # Calculate the bounding box coordinates
            left = min(x_coords)
            upper = min(y_coords)
            right = max(x_coords)
            lower = max(y_coords)

        else:
            print(f"Unsupported shape type: {shape['shape_type']}. Skipping this shape.")
            continue

        # Crop the image using the bounding box
        cropped_image = image.crop((left, upper, right, lower))

        # Generate output filename and handle naming conflicts
        label = shape['label']
        base_output_path = os.path.join(output_dir, f"{original_base_name}_{label}_{int(left)}_{int(upper)}.jpg")
        output_path = base_output_path

        # Handle naming conflicts by appending numbers in brackets
        counter = 1
        while os.path.exists(output_path):
            output_path = os.path.splitext(base_output_path)[0] + f"({counter})" + os.path.splitext(base_output_path)[1]
            counter += 1
        
        cropped_image.save(output_path)

def process_directory(image_dir, json_dir, output_base_dir):
    # Create subdirectories for 1, 2, 5 and 9 classes
    class_names = ["1", "2", "5", "9"]
    class_dirs = {class_name: os.path.join(output_base_dir, class_name) for class_name in class_names}
    for class_dir in class_dirs.values():
        os.makedirs(class_dir, exist_ok=True)

    # Get lists of all image and json files
    image_files = [f for f in os.listdir(image_dir) if f.endswith('.jpg') or f.endswith('.jpeg')]
    json_files = [f for f in os.listdir(json_dir) if f.endswith('.json')]

    # Check if no images were found
    if len(image_files) == 0:
        print(f"ERROR: No image files (jpg/jpeg) found in directory: {image_dir}")
        return

    # Check if no JSON files were found
    if len(json_files) == 0:
        print(f"ERROR: No JSON files found in directory: {json_dir}")
        return

    # Track the number of successfully processed images
    processed_count = 0

    for image_file in image_files:
        base_name = os.path.splitext(image_file)[0]
        json_file = f"{base_name}.json"

        if json_file in json_files:
            image_path = os.path.join(image_dir, image_file)
            json_path = os.path.join(json_dir, json_file)

            # Determine the class of the image based on its name
            if image_file.startswith("T") and len(image_file) > 1:
                class_digit = image_file[1]  # Get the character right after "T"
                if class_digit in class_names:
                    output_dir = class_dirs[class_digit]
                else:
                    print(f"Warning: Image {image_file} has unsupported class prefix '{class_digit}'. Skipping.")
                    continue
            else:
                print(f"Warning: Image {image_file} does not start with 'T' or has invalid format. Skipping.")
                continue

            print(f"Processing image: {image_path} with JSON: {json_path}")

            crop_objects_from_image(image_path, json_path, output_dir)
            processed_count += 1
        else:
            print(f"Warning: No JSON file found for image: {image_file}")

    # Report final processing status
    if processed_count == 0:
        print(f"ERROR: No images could be processed. Check if image names match their corresponding JSON files.")
    else:
        print(f"Successfully processed {processed_count} out of {len(image_files)} images.")

# Example directory paths
# default : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\TrainingStepSet\\jpg\\' ; # alt = '' ;
# notes : manual annotations, 100% ;
# example 01 : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\task-04_dataset\\HM_aligned_to_SC_renamed_files_manual_labels_T_(new_OM_images)'
# notes : automated annotations, 100% ;
# example 02 : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\task-04_dataset\\HM_aligned_to_SC_renamed_files_automated_labels_T'
# notes : manual annotations, 125% ;
# example 03 : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\task-04_dataset\\25_pc_expanded_manual_labels_with_aligned_HM_images_T'
# notes : automated annotations, 125% ;
# example 04 : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\task-04_dataset\\25_pc_expanded_automated_labels_with_aligned_HM_images_T'
# notes : manual annotations, 150% ;
# example 05 : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\task-04_dataset\\50_pc_expanded_manual_labels_with_aligned_HM_images_T'
# notes : automated annotations, 150% ;
# example 06 : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\task-04_dataset\\50_pc_expanded_automated_labels_with_aligned_HM_images_T'
# notes : ?
# example 07 : ''
image_dir = 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\task-04_dataset\\50_pc_expanded_automated_labels_with_aligned_HM_images_T' 
# default = 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\TrainingStepSet\\new-training-labels-json\\' ; # alt = '' ;
# notes : manual annotations, 100% ;
# example 01 : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\task-04_dataset\\HM_aligned_to_SC_renamed_files_manual_labels_T_(new_OM_images)'
# notes : automated annotations, 100% ;
# example 02 : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\task-04_dataset\\HM_aligned_to_SC_renamed_files_automated_labels_T'
# notes : manual annotations, 125% ;
# example 03 : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\task-04_dataset\\25_pc_expanded_manual_labels_with_aligned_HM_images_T'
# notes : automated annotations, 125% ;
# example 04 : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\task-04_dataset\\25_pc_expanded_automated_labels_with_aligned_HM_images_T'
# notes : manual annotations, 150% ;
# example 05 : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\task-04_dataset\\50_pc_expanded_manual_labels_with_aligned_HM_images_T'
# notes : automated annotations, 150% ;
# example 06 : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\task-04_dataset\\50_pc_expanded_automated_labels_with_aligned_HM_images_T'
# notes : ?
# example 07 : ''
json_dir = 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\task-04_dataset\\50_pc_expanded_automated_labels_with_aligned_HM_images_T' 
# default : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\new_work_02\\vet_images_sliced\\TrainingStepSet\\' ; # alt = '' ;
# notes : manual annotations, 100% ;
# example 01 : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\04th-setup_task-04_new-work-03\\vet_images_sliced\\TrainingStepSet_manual_labels_T_640_100-pc'
# notes : automated annotations, 100% ;
# example 02 : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\04th-setup_task-04_new-work-03\\vet_images_sliced\\TrainingStepSet_automated_labels_T_640_100-pc'
# notes : manual annotations, 125% ;
# example 03 : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\04th-setup_task-04_new-work-03\\vet_images_sliced\\TrainingStepSet_manual-labels_T_640_125-pc'
# notes : automated annotations, 125% ;
# example 04 : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\04th-setup_task-04_new-work-03\\vet_images_sliced\\TrainingStepSet_automated-labels_T_640_125-pc'
# notes : manual annotations, 150% ;
# example 05 : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\04th-setup_task-04_new-work-03\\vet_images_sliced\\TrainingStepSet_manual-labels_T_640_150-pc'
# notes : automated annotations, 150% ;
# example 06 : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\04th-setup_task-04_new-work-03\\vet_images_sliced\\TrainingStepSet_automated-labels_T_640_150-pc'
# notes : ?
# example 07 : ''
output_base_dir = 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\04th-setup_task-04_new-work-03\\vet_images_sliced\\TrainingStepSet_automated-labels_T_640_150-pc' 

process_directory(image_dir, json_dir, output_base_dir)

#image_path = 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\TrainingSet\\jpg\\T1(2)TAPE.jpg' ; # default = '/Users/gustavszviedris/Desktop/vet_images/Training/9/T95_28.02.jpg' ; alt = 'C:\\projects\\SlideScanner2_28.2.2024\\T95_28.02.jpg' ; 
#json_path = 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\TrainingSet_(older-data)\\labeled\\T1(2)TAPE.json' ; # default = '/Users/gustavszviedris/Desktop/Labeled_Slides/Training/9/T95_28.02.json' ; alt = 'C:\\projects\\Labeled_Slides\\T95_28.02.json' ; 
#output_dir = 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\new_work_02\\vet_images_sliced\\TrainingSet\\T1(2)TAPE\\' ; # default = '/Users/gustavszviedris/Desktop/vet_images_sliced/Training/9/' ; alt = 'E:\\-_EDI_-\\notes\\havetai+vetcyto\\vet_images_sliced\\TrainingStep\\9' ; 

#crop_objects_from_image(image_path, json_path, output_dir)


# Testing Step - Cutting Code.

In [ ]:
import json
from PIL import Image
from tqdm import tqdm
import os
import shutil

def crop_objects_from_image(image_path, json_path, output_dir):
    # Null address check
    if not (os.path.exists(image_path)):
        print("The jpeg path is empty! Check for mistakes! \ninput arguments: \n", "image_path =", image_path, "\n", "json_path =", json_path, "\n", "output_dir =", output_dir, "\n")
        return
    if not (os.path.exists(json_path)):
        print("The json path is empty! Check for mistakes! \ninput arguments: \n", "image_path =", image_path, "\n", "json_path =", json_path, "\n", "output_dir =", output_dir, "\n")
        return

    # Check if output directory exists and handle accordingly
    if os.path.exists(output_dir):
        print(f"The output directory '{output_dir}' already exists. It'll be replaced.")
        shutil.rmtree(output_dir)
    print(f"Creating output directory: '{output_dir}'")
    os.makedirs(output_dir, exist_ok=True)

    # Load the JSON file containing object annotations
    with open(json_path, 'r') as f:
        data = json.load(f)

    # Increase the maximum allowed image size by modifying the MAX_IMAGE_PIXELS constant in PIL to reduce zip bomb warnings
    Image.MAX_IMAGE_PIXELS = 250000000 

    # Open the image file
    image = Image.open(image_path)

    # Iterate through each shape in the JSON data, with a progress bar
    for shape in tqdm(data['shapes'], desc="Cropping objects"):
        points = shape['points']
        
        if shape['shape_type'] == 'rectangle':
            # Extract rectangle coordinates
            x1, y1 = points[0]
            x2, y2 = points[1]

            # Calculate the bounding box coordinates
            left = min(x1, x2)
            upper = min(y1, y2)
            right = max(x1, x2)
            lower = max(y1, y2)

        elif shape['shape_type'] == 'polygon':
            # Extract polygon coordinates
            x_coords = [point[0] for point in points]
            y_coords = [point[1] for point in points]

            # Calculate the bounding box coordinates
            left = min(x_coords)
            upper = min(y_coords)
            right = max(x_coords)
            lower = max(y_coords)

        else:
            print(f"Unsupported shape type: {shape['shape_type']}. Skipping this shape.")
            continue

        # Crop the image using the bounding box
        cropped_image = image.crop((left, upper, right, lower))

        # Generate output filename and save the cropped image
        label = shape['label']
        output_path = f"{output_dir}/{label}_{int(left)}_{int(upper)}.jpg"
        cropped_image.save(output_path)

def process_directory(image_dir, json_dir, output_base_dir):
    # Get lists of all image and json files
    image_files = [f for f in os.listdir(image_dir) if f.endswith('.jpg') or f.endswith('.jpeg')]
    json_files = [f for f in os.listdir(json_dir) if f.endswith('.json')]

    for image_file in image_files:
        base_name = os.path.splitext(image_file)[0]
        json_file = f"{base_name}.json"

        if json_file in json_files:
            image_path = os.path.join(image_dir, image_file)
            json_path = os.path.join(json_dir, json_file)
            output_dir = os.path.join(output_base_dir, base_name)

            print(f"Processing image: {image_path} with JSON: {json_path}")

            crop_objects_from_image(image_path, json_path, output_dir)
        else:
            print(f"Warning: No JSON file found for image: {image_path}")

# Example directory paths
image_dir = 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\TestingStepSet\\jpg\\' ; # alt = '' ;
json_dir = 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\TestingStepSet\\labeled\\' ; # alt = '' ;
output_base_dir = 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\new_work_02\\vet_images_sliced\\TestingStepSet\\' ; # alt = '' ;

process_directory(image_dir, json_dir, output_base_dir)

#image_path = 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\TrainingSet\\jpg\\T1(2)TAPE.jpg' ; # default = '/Users/gustavszviedris/Desktop/vet_images/Training/9/T95_28.02.jpg' ; alt = 'C:\\projects\\SlideScanner2_28.2.2024\\T95_28.02.jpg' ; 
#json_path = 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\TrainingSet_(older-data)\\labeled\\T1(2)TAPE.json' ; # default = '/Users/gustavszviedris/Desktop/Labeled_Slides/Training/9/T95_28.02.json' ; alt = 'C:\\projects\\Labeled_Slides\\T95_28.02.json' ; 
#output_dir = 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\new_work_02\\vet_images_sliced\\TrainingSet\\T1(2)TAPE\\' ; # default = '/Users/gustavszviedris/Desktop/vet_images_sliced/Training/9/' ; alt = 'E:\\-_EDI_-\\notes\\havetai+vetcyto\\vet_images_sliced\\TrainingStep\\9' ; 

#crop_objects_from_image(image_path, json_path, output_dir)


# Training Step - Splitting Code.

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split
from tqdm import tqdm

def create_directories(output_path, classes):
    # Create train, val, and test directories for each class
    for split in ['train', 'val', 'test']:
        for class_name in classes:
            os.makedirs(os.path.join(output_path, split, class_name), exist_ok=True)

def split_and_copy_files(source_path, output_path, class_name, train_size=0.7, val_size=0.15, test_size=0.15):
    class_path = os.path.join(source_path, class_name)

    if not os.path.exists(class_path):
        print(f"WARNING: Directory does not exist: {class_path}")
        return False

    files = os.listdir(class_path)
    files = [f for f in files if os.path.isfile(os.path.join(class_path, f))]
    
    print(f"Class {class_name}: {len(files)} files found")

    if not files:
        print(f"WARNING: No files found in class {class_name}")
        return False

    # Split files into train, validation, and test sets
    train_files, temp_files = train_test_split(files, test_size=(val_size + test_size), random_state=42)
    val_files, test_files = train_test_split(temp_files, test_size=test_size/(val_size + test_size), random_state=42)

    # Copy files without progress bars
    splits_count = {"train": 0, "val": 0, "test": 0}
    
    for split, file_list in [('train', train_files), ('val', val_files), ('test', test_files)]:
        for file in file_list:
            try:
                src = os.path.join(class_path, file)
                dst = os.path.join(output_path, split, class_name, file)
                os.makedirs(os.path.dirname(dst), exist_ok=True)
                shutil.copy(src, dst)
                splits_count[split] += 1
            except Exception as e:
                print(f"WARNING: Error copying file {file}: {str(e)}")
    
    # Report splits
    print(f"Class {class_name} split: train={splits_count['train']}, val={splits_count['val']}, test={splits_count['test']}")
    
    return True

def split_dataset(source_folder, output_folder):
    # Get all class names
    try:
        class_names = [d for d in os.listdir(source_folder) if os.path.isdir(os.path.join(source_folder, d))]
    except Exception as e:
        print(f"WARNING: Could not access source folder: {str(e)}")
        return
    
    if not class_names:
        print("WARNING: No classes found in source folder")
        return
    
    # Count total files
    total_files = 0
    for class_name in class_names:
        class_path = os.path.join(source_folder, class_name)
        if os.path.exists(class_path):
            files = [f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))]
            total_files += len(files)
    
    print(f"Total files found: {total_files}")
    print(f"Classes found: {', '.join(class_names)}")

    # Check if output directory exists and handle accordingly
    if os.path.exists(output_folder):
        print(f"The output directory '{output_folder}' already exists. It'll be replaced.")
        shutil.rmtree(output_folder)
    print(f"Creating output directory: '{output_folder}'")
    os.makedirs(output_folder, exist_ok=True)

    # Create the necessary directory structure
    create_directories(output_folder, class_names)

    # Process each class with a single progress bar
    with tqdm(class_names, desc="Processing classes", unit="class", ncols=100) as pbar:
        for class_name in pbar:
            pbar.set_description(f"Processing class: {class_name}")
            try:
                split_and_copy_files(source_folder, output_folder, class_name)
            except Exception as e:
                print(f"WARNING: Processing interrupted for class {class_name}: {str(e)}")

# Example usage
# default = '/Users/gustavszviedris/Desktop/vet_images_sliced_copy/Training' ; 
# alt 01 = 'E:\\-_EDI_-\\notes\\havetai+vetcyto\\vet_images_sliced\\Training' ; 
# alt 02 = 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\new_work_02\\vet_images_sliced\\TrainingStepSet'
# notes : manual annotations, 100% ;
# example 01 : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\04th-setup_task-04_new-work-03\\vet_images_sliced\\TrainingStepSet_manual-labels_T_640_100-pc'
# notes : automated annotations, 100% ;
# example 02 : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\04th-setup_task-04_new-work-03\\vet_images_sliced\\TrainingStepSet_automated-labels_T_640_100-pc'
# notes : manual annotations, 125% ;
# example 03 : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\04th-setup_task-04_new-work-03\\vet_images_sliced\\TrainingStepSet_manual-labels_T_640_125-pc'
# notes : automated annotations, 125% ;
# example 04 : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\04th-setup_task-04_new-work-03\\vet_images_sliced\\TrainingStepSet_automated-labels_T_640_125-pc'
# notes : manual annotations, 150% ;
# example 05 : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\04th-setup_task-04_new-work-03\\vet_images_sliced\\TrainingStepSet_manual-labels_T_640_150-pc'
# notes : automated annotations, 150% ;
# example 06 : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\04th-setup_task-04_new-work-03\\vet_images_sliced\\TrainingStepSet_automated-labels_T_640_150-pc'
# notes : ?
# example 07 : ''
source_folder = 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\04th-setup_task-04_new-work-03\\vet_images_sliced\\TrainingStepSet_automated-labels_T_640_150-pc' ; 
# default = '/Users/gustavszviedris/Desktop/vet_images_sliced_split' ; 
# alt 01 = 'E:\\-_EDI_-\\notes\\havetai+vetcyto\\vet_images_sliced_split' ; 
# alt 02 = 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\new_work_02\\vet_images_sliced_split\\TrainingStepSet'
# notes : manual annotations, 100% ;
# example 01 : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\04th-setup_task-04_new-work-03\\vet_images_sliced_split\\TrainingStepSet_manual-labels_T_640_100-pc'
# notes : automated annotations, 100% ;
# example 02 : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\04th-setup_task-04_new-work-03\\vet_images_sliced_split\\TrainingStepSet_automated-labels_T_640_100-pc'
# notes : manual annotations, 125% ;
# example 03 : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\04th-setup_task-04_new-work-03\\vet_images_sliced_split\\TrainingStepSet_manual-labels_T_640_125-pc'
# notes : automated annotations, 125% ;
# example 04 : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\04th-setup_task-04_new-work-03\\vet_images_sliced_split\\TrainingStepSet_automated-labels_T_640_125-pc'
# notes : manual annotations, 150% ;
# example 05 : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\04th-setup_task-04_new-work-03\\vet_images_sliced_split\\TrainingStepSet_manual-labels_T_640_150-pc'
# notes : automated annotations, 150% ;
# example 06 : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\04th-setup_task-04_new-work-03\\vet_images_sliced_split\\TrainingStepSet_automated-labels_T_640_150-pc'
# notes : ?
# example 07 : ''
output_folder = 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\04th-setup_task-04_new-work-03\\vet_images_sliced_split\\TrainingStepSet_automated-labels_T_640_150-pc' ; 

split_dataset(source_folder, output_folder)


# Testing Step - Splitting Code.

In [ ]:
# NOT NEEDED


# notes: cleaning up garbage files (such as discarded results)

In [ ]:
import shutil

# Navigate to the data set folder & delete any read-only folders, in case that they get created by accident
! echo "# test 01" && cd "C:\\Users\\praam\\Desktop\\havetai+vetcyto\\04th_task-04_new-work-03\\February15.v1i.yolov8" && dir
#shutil.rmtree("C:\\Users\\praam\\Desktop\\havetai+vetcyto\\04th-setup_task-04_new-work-03\\February15.v1i.yolov8\\splitting_output_automated_labels_T_640\\results")
#shutil.rmtree("C:\\Users\\praam\\Desktop\\havetai+vetcyto\\04th-setup_task-04_new-work-03\\February15.v1i.yolov8\\splitting_output_automated_labels_T_640\\runs")
